In [2]:
import cv2
import PIL.Image as Image
import matplotlib.pyplot as plt
import requests

import numpy as np
from PIL import Image

im = cv2.imread("./parts.jpg")

In [3]:
#VELIKI
params1=cv2.SimpleBlobDetector_Params()

params1.filterByArea=True
params1.minArea = 3100
params1.maxArea = 10000  

params1.filterByCircularity = True
params1.minCircularity = 0.6

params1.filterByConvexity = False
params1.filterByInertia = False
params1.filterByColor = False

detector1=cv2.SimpleBlobDetector_create(params1)


#MALI
params2=cv2.SimpleBlobDetector_Params()

params2.filterByArea=True
params2.minArea = 2000
params2.maxArea = 3000

params2.filterByCircularity = True
params2.minCircularity = 0.6

params2.filterByConvexity = False
params2.filterByInertia = False
params2.filterByColor = False

detector2=cv2.SimpleBlobDetector_create(params2)

In [7]:
resTrack = (1920,1080)
resWrite = (1920,1080)

vid = cv2.VideoCapture(1)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, resTrack[0])
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, resWrite[1])

while True:
    ret, frame = vid.read()
    if not ret:
        break

    # Pretvorba v HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    lower_color1 = np.array([0,80,80])
    upper_color1 = np.array([20,80,80])
    lower_color2 = np.array([160,80,80])
    upper_color2 = np.array([180,255,255])
    


    mask1 = cv2.inRange(hsv, lower_color1, upper_color1)
    mask2 = cv2.inRange(hsv, lower_color2, upper_color2)

    mask = cv2.bitwise_or(mask1, mask2)
    
    cv2.namedWindow("frame",cv2.WINDOW_NORMAL)
    cv2.resizeWindow("frame",1280, 720)
    cv2.imshow('frame', mask)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
vid.release()
cv2.destroyAllWindows()

In [11]:
resTrack = (1920,1080)
resWrite = (1920,1080)

vid = cv2.VideoCapture(0)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, resTrack[0])
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, resWrite[1])

while True:
    ret, frame = vid.read()
    if not ret:
        break

    # Pretvorba v HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Filtriramo svetle barve (npr. bele)
    #lower_color = np.array([0, 0, 200])
    #upper_color = np.array([180, 30, 255])
    #mask = cv2.inRange(hsv, lower_color, upper_color)
    
    
    
    lower_color1 = np.array([0,80,80])
    upper_color1 = np.array([20,80,80])
    lower_color2 = np.array([160,80,80])
    upper_color2 = np.array([180,255,255])

    mask1 = cv2.inRange(hsv, lower_color1, upper_color1)
    mask2 = cv2.inRange(hsv, lower_color2, upper_color2)

    mask = cv2.bitwise_or(mask1, mask2)


    # Detekcija točk
    keypoints1 = detector1.detect(mask)
    keypoints2 = detector2.detect(mask)
    
    veliki = []
    mali = []
    
    for keyPoint in keypoints1:
        u=keyPoint.pt[1]
        v=keyPoint.pt[0]

        scale=1 
        x=(v)*scale       
        y=(u)*scale
    
        veliki.append([x,y])

    
    for keyPoint in keypoints2:
        u=keyPoint.pt[1]
        v=keyPoint.pt[0]

        scale=1 
        x=(v)*scale       
        y=(u)*scale
    
        mali.append([x,y])
    
    
    koti = []
    for (x1, y1), (x2, y2) in zip(veliki, mali):  
        dx = x2 - x1
        dy = y2 - y1
    
        kot_rad = np.arctan2(dy, dx)     
        kot_deg = np.degrees(kot_rad)    

        #print(kot_deg)
        koti.append(kot_rad)
        #print(kot_rad)
    
    popravki = []
    for kot in koti:
        popravek = kot -0.6071841235083298
        popravki.append(popravek*180/np.pi)
        #print(popravek)
    
    
    
    
    # Narišemo keypointe z radijem glede na velikost
    for kp in keypoints1:
        x, y = np.int32(kp.pt)
        radius = int(kp.size / 2)  # velikost keypointa / 2 = radij
        cv2.circle(frame, (x, y), radius=radius, color=(0, 0, 255), thickness=2)
    for kp in keypoints2:
        x, y = np.int32(kp.pt)
        radius = int(kp.size / 2)  # velikost keypointa / 2 = radij
        cv2.circle(frame, (x, y), radius=radius, color=(0, 0, 255), thickness=2)
    
    #izhodišče K.S.
    cv2.circle(frame, (1430, 180), radius=2, color=(0, 0, 255), thickness=2)
    
    for (x1, y1), (x2, y2) in zip(veliki, mali):
        cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 3)
    
    cv2.namedWindow("frame",cv2.WINDOW_NORMAL)
    cv2.resizeWindow("frame",1280, 720)
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    #Komunikacija
    #for i in range(len(veliki)):
        #r = requests.get('http://192.161.1.100/KAREL/setprc?str_coord1={}&str_coord2={}&str_coord3=0.0&str_coord4=0.0&str_coord5=0.0&str_coord6={}'.format(veliki[i][0],veliki[i][1],popravek[i]))
        #zapiranje gripperja
        #r = requests.get('http://192.168.1.100/KAREL/ros_cgio?io_op=write&io_type=9&io_idx=1&io_val=0')
        #odpiranje gripperja
        
        #r = requests.get('http://192.168.1.100/KAREL/ros_cgio?io_op=write&io_type=9&io_idx=1&io_val=1')
vid.release()
cv2.destroyAllWindows()

In [32]:
mali

[[2170.7936401367188, 1533.5211791992188]]

In [33]:
veliki

[[2211.0239868164062, 1624.544677734375]]